## Pipelines
Dataset: Home prices in Melbourne, Australia

In [1]:
import pandas as pd
price_data = pd.read_csv('Datasets/melb_data.csv')

y = price_data.Price
X = price_data.drop('Price', axis=1)

from sklearn.model_selection import train_test_split
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y , train_size=0.8, test_size=0.2)

In [2]:
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [3]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
1800,t,SP,Southern Metropolitan,4,8.9,3162.0,4.0,3.0,2.0,812.0,NaN,NaN,-37.88640,145.03260,2379.0
7895,h,S,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,1011.0,NaN,NaN,-37.71150,144.91200,8870.0
9897,h,S,Northern Metropolitan,3,16.1,3088.0,3.0,1.0,2.0,1020.0,NaN,NaN,-37.70821,145.10405,8524.0
7578,u,S,Southern Metropolitan,2,10.7,3187.0,2.0,1.0,3.0,238.0,NaN,NaN,-37.91260,145.01050,6938.0
7505,t,S,Southern Metropolitan,3,13.0,3204.0,3.0,2.0,2.0,334.0,NaN,2006.0,-37.92840,145.04280,6795.0


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [5]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [6]:
from sklearn.metrics import mean_absolute_error
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

my_pipeline.fit(X_train, y_train)
preds = my_pipeline.predict(X_test)

score = mean_absolute_error(y_test, preds)
print('MAE:', score)

MAE: 159963.61156217125
